In [120]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [121]:
### Load the trained model, scaler pickle,onehot
model=load_model('ann_model.h5')

## load the encoder and scaler
with open('ann_onehot_encoder_cropName.pkl','rb') as file:
    onehot_encoder_cropName=pickle.load(file)

with open('ann_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [122]:
df1 = pd.read_csv("processed_df.csv")
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df1 = df1[df1["targetYield"]!=0]

df2 = pd.read_csv("df_with_predicted_TargetYields.csv")
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

df = pd.concat([df1, df2])
df.columns

# a = df1.iloc[3].to_dict()
# a

Index(['previousYield', 'organicCarbon', 'pH', 'phosphorus', 'cu', 'fe',
       'potassium', 'targetYield', 'boron', 'zn', 'sulphur',
       'electricalConductivity', 'nitrogen',
       'cropNameCommon_chilli (irrigated)', 'cropNameCommon_chilli (rainfed)',
       'cropNameCommon_foxtail millet', 'cropNameCommon_green gram',
       'cropNameCommon_groundnut  (rainfed)',
       'cropNameCommon_sunflower (irrigated)',
       'cropNameCommon_sunflower (rainfed)',
       'previousCropName_chilli (rainfed)', 'previousCropName_coriander',
       'previousCropName_cotton', 'previousCropName_foxtail millet',
       'previousCropName_green gram', 'previousCropName_groundnut (irrigated)',
       'previousCropName_maize', 'previousCropName_sorghum (irrigated)',
       'previousCropName_sorghum/jowar (irrigated)',
       'previousCropName_sunflower (irrigated)',
       'previousCropName_sunflower (rainfed)', 'previousCropName_tomato'],
      dtype='object')

In [123]:
# Example input data
input_data = {'previousYield': 0.0,
 'organicCarbon': 0.66,
 'pH': 8.07,
 'phosphorus': 38.0,
 'cu': 0.1,
 'fe': 10.42,
 'potassium': 176.0,
 'boron': 0.17,
 'zn': 0.32,
 'sulphur': 8.5,
 'electricalConductivity': 0.42,
 'nitrogen': 0.0,
 'previousCropName':"chilli (rainfed)",
 'targetYield': 19140.09224384223
 }

In [124]:
input_df=pd.DataFrame([input_data])
input_df

,previousYield,organicCarbon,pH,phosphorus,cu,fe,potassium,boron,zn,sulphur,electricalConductivity,nitrogen,previousCropName,targetYield
0,0.0,0.66,8.07,38.0,0.1,10.42,176.0,0.17,0.32,8.5,0.42,0.0,chilli (rainfed),19140.092244


In [125]:
print(input_data['previousCropName'])

chilli (rainfed)


In [126]:
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(drop='first')
# encoded_cols = encoder.transform([[input_data['previousCropName']]]).toarray()
# encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(['previousCropName']))

all_categories = [
    'previousCropName_chilli (rainfed)', 'previousCropName_coriander',
    'previousCropName_cotton', 'previousCropName_foxtail millet',
    'previousCropName_green gram', 'previousCropName_groundnut (irrigated)',
    'previousCropName_maize', 'previousCropName_sorghum (irrigated)',
    'previousCropName_sorghum/jowar (irrigated)',
    'previousCropName_sunflower (irrigated)',
    'previousCropName_sunflower (rainfed)', 'previousCropName_tomato'
]

# Reshape for fitting (each category should be a 2D array with one column)
all_categories = [[category] for category in all_categories]

# Initialize and fit the OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(all_categories)

# Transform new input data (example input)
input_data = {'previousCropName': 'previousCropName_green gram'}
encoded_cols = encoder.transform([[input_data['previousCropName']]])

with open('ann_onehot_encoder_prevCropName.pkl','wb') as file:
    pickle.dump(encoder,file)

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(['previousCropName']))

encoded_df

,previousCropName_previousCropName_coriander,previousCropName_previousCropName_cotton,previousCropName_previousCropName_foxtail millet,previousCropName_previousCropName_green gram,previousCropName_previousCropName_groundnut (irrigated),previousCropName_previousCropName_maize,previousCropName_previousCropName_sorghum (irrigated),previousCropName_previousCropName_sorghum/jowar (irrigated),previousCropName_previousCropName_sunflower (irrigated),previousCropName_previousCropName_sunflower (rainfed),previousCropName_previousCropName_tomato
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# # cols_to_encode = ["previousCropName"]
# # encoded_cols = onehot_encoder.transform(input_df[cols_to_encode])
# # encoded_df = pd.DataFrame(encoded_cols, columns=onehot_encoder.get_feature_names_out(cols_to_encode))

# encoded_cols = onehot_encoder.transform([[input_data['previousCropName']]]).toarray()
# print("encoded_cols ", encoded_cols)

# encoded_df = pd.DataFrame(encoded_cols, columns=onehot_encoder.get_feature_names_out(['previousCropName']))
# print(onehot_encoder.get_feature_names_out(['previousCropName']))

# Concatenate the encoded columns with the original numeric columns
final_df = pd.concat([input_df.drop(["previousCropName"], axis=1), encoded_df], axis=1)
final_df

,previousYield,organicCarbon,pH,phosphorus,cu,fe,potassium,boron,zn,sulphur,...,previousCropName_previousCropName_cotton,previousCropName_previousCropName_foxtail millet,previousCropName_previousCropName_green gram,previousCropName_previousCropName_groundnut (irrigated),previousCropName_previousCropName_maize,previousCropName_previousCropName_sorghum (irrigated),previousCropName_previousCropName_sorghum/jowar (irrigated),previousCropName_previousCropName_sunflower (irrigated),previousCropName_previousCropName_sunflower (rainfed),previousCropName_previousCropName_tomato
0,0.0,0.66,8.07,38.0,0.1,10.42,176.0,0.17,0.32,8.5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
final_df.columns

Index(['previousYield', 'organicCarbon', 'pH', 'phosphorus', 'cu', 'fe',
       'potassium', 'boron', 'zn', 'sulphur', 'electricalConductivity',
       'nitrogen', 'targetYield',
       'previousCropName_previousCropName_coriander',
       'previousCropName_previousCropName_cotton',
       'previousCropName_previousCropName_foxtail millet',
       'previousCropName_previousCropName_green gram',
       'previousCropName_previousCropName_groundnut (irrigated)',
       'previousCropName_previousCropName_maize',
       'previousCropName_previousCropName_sorghum (irrigated)',
       'previousCropName_previousCropName_sorghum/jowar (irrigated)',
       'previousCropName_previousCropName_sunflower (irrigated)',
       'previousCropName_previousCropName_sunflower (rainfed)',
       'previousCropName_previousCropName_tomato'],
      dtype='object')

In [129]:
scaler

StandardScaler()

In [130]:
expected_columns = scaler.feature_names_in_
final_df_aligned = final_df.reindex(columns=expected_columns, fill_value=0)
input_scaled = scaler.transform(final_df_aligned)

In [131]:
## Scaling the input data

input_scaled=scaler.transform(final_df_aligned)
input_scaled

array([[-0.06527532, -0.31317994,  1.33676307, -0.12817089, -1.32725343,
         1.20688839, -0.04246689,  0.44880403, -1.13540255, -0.92906827,
        -0.76552817,  0.44178301,  0.        , -0.82808035, -0.01421961,
        -0.10508846, -0.02011159, -0.02596746, -0.37796447, -0.04700964,
        -0.02172374, -0.01835867, -0.24164887, -0.02463406, -0.02011159]])

In [132]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 87ms/step


array([[0.2406885 , 0.39142168, 0.08949543, 0.02848419, 0.0500392 ,
        0.10600755, 0.09386344]], dtype=float32)

In [133]:
prediction_proba = prediction[0][0]
prediction_proba

0.2406885

In [136]:
import numpy as np

predicted_class_index = np.argmax(prediction)
print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted probability: {prediction[0][predicted_class]}")

predicted_label = onehot_encoder_cropName.categories_[0][predicted_class_index]

print(f"Predicted label: {predicted_label}")

Predicted class index: 1
Predicted probability: 0.39142167568206787
Predicted label: chilli (irrigated)
